Enter verification code: 4/1AbUR2VMHv2qobAL5gGblUlynmWwrvoBShsI7192b-tb9UWfFR5GBgHuMjIQ

Successfully saved authorization token.


In [1]:
import argparse
import yaml
from ee_download_workflow import download_project
from utils import get_project_data

with open("../config/gee/project_timeserie.yaml") as f: 
    cfg = yaml.safe_load(f)

download_project(**cfg, verbose=True)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=%E2%80%9C757499213145-tufpdalliu6ef9s5cdkaet2ra7bfobo8.apps.googleusercontent.com%E2%80%9D&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=%E2%80%9Chttps%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%22&access_type=offline&response_type=code

Enter verification code: 4/1AbUR2VMHv2qobAL5gGblUlynmWwrvoBShsI7192b-tb9UWfFR5GBgHuMjIQ


AuthenticationError: OAuth2 code exchange failed: invalid_clientThe OAuth client was not found.

In [6]:
import ee
ee.Initialize()
import yaml
import time
import rasterio
import rasterio.warp 
import rasterio.features
import fiona
from datetime import datetime, timedelta
import numpy as np
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")

In [27]:
def pjoin(*subs): return Path(os.path.join(*subs))
def parse_date(date_str) -> datetime: return datetime.strptime(date_str, '%Y%m%d')
def parse_gt_date(date_str) -> datetime: 
    # there are 2 gt date formats
    try: return datetime.strptime(date_str, "%Y-%m-%d")
    except: return datetime.strptime(date_str, "%Y/%m/%d")

# Load project data

- Given paths and projet_id:
    - gt_file
    - gt_labels
    - gt_date
    - polygon
    - bounds
    - gt_crs
    - aoi_df

In [ ]:
class ProjectInfo():
    def __init__(self, gt_file, gt, gt_date, gt_crs, polygon, bounds):
        self.gt_file = gt_file
        self.

In [29]:
def configure(project_id, root="/scratch/ewalt/pdm/rs-uncertainty/"):
    return {
        "project_id": project_id,
        "gt_dir": pjoin(root, "assets/data/preprocessed"),
        "gt_data_bands": [1, 2, 3, 4, 5],
        "shapefile_paths": [
            pjoin(root, p) for p in 
            ['assets/data/NHM_projectDekning_AOI_edit2015_V2.shp', 'assets/data/ALS_projects_Dz_all_norway.shp']
        ],
    }

In [28]:
def get_project_data(
    project_id,
    gt_dir,
    gt_data_bands,
    shapefile_paths,
    return_df=True
):
    # Get gt
    gt_file = rasterio.open(pjoin(gt_dir, project_id+".tif"))
    gt = gt_file.read(gt_data_bands)
    bounds = gt_file.bounds
    # Load shapefiles
    project_shape_collections = [fiona.open(p) for p in shapefile_paths]
    # create the shape ("polygon") associated to the project 
    polygon, gt_crs, gt_date = None, None, None
    for collection in project_shape_collections:
        try:
            polygon = [s['geometry'] for s in collection if s['properties']['kv_id'] == int(project_id)][0]
            crs = collection.crs
            gt_date = [s["properties"]["PUB_DATO"] for s in collection if s['properties']['kv_id'] == int(project_id)][0]
            gt_date = parse_gt_date(gt_date)
            break
        except IndexError: pass 
    if polygon is None: print("No polygon found")
    polygon = rasterio.warp.transform_geom(src_crs=crs, dst_crs=gt_file.crs, geom=polygon)
    return gt_file, gt, gt_date, gt_crs, polygon["coordinates"], bounds

In [30]:
project_id = "498"
gt_file, gt, gt_date, gt_crs, polygon, bounds = get_project_data(**configure(project_id))

In [32]:
bounds.left

538395.6051443298

In [49]:
polygon["type"]

'Polygon'

In [ ]:
def mergeByDate(imgCol):
    '''
    function that merges images together that have the same date. 
    adapted from: https://gis.stackexchange.com/questions/372744/how-to-create-a-loop-which-creates-single-mosaic-images-for-each-day-from-a-sel

    PARAMETERS
    imgCol: [ee.ImageCollection] mandatory value that specifies the image collection to merge by dates with.

    RETURNS 
    ee.ImageCollection where images of the same day are mosaiced together
    '''    
    #Convert the image collection to a list.
    imgList = imgCol.toList(imgCol.size())
    
    # Driver function for mapping the unique dates
    def uniqueDriver(image):
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    uniqueDates = imgList.map(uniqueDriver).distinct()

    # Driver function for mapping the moasiacs
    def mosaicDriver(date):
        date = ee.Date(date)
        
        image = (imgCol
               .filterDate(date, date.advance(1, "day")) #or (date.advance(-1, "day"), date.advance(1, "day"))?
               .mosaic())
        
        return image.set("system:time_start", date.millis(), 
                        "system:date", date.format("YYYY-MM-dd"),
                        "system:id", date.format("YYYY-MM-dd"))
    
    mosaicImgList = uniqueDates.map(mosaicDriver)
    
    return ee.ImageCollection(mosaicImgList)

def download_local(driverfolderid, localdir, start, verbose=False):
    while still_going(start):
        time.sleep(1)
        downloadasgo(drivefolderid, localdir, verbose=verbose)
    downloadasgo(drivefolderid, localdir, verbose=verbose)

def download_image(
    image: ee.Image,
    project_id: str,
):
    # Make filename
    # define task
    # start task
    raise NotImplementedError()

def download_project(
    localdir: str,
    project_id: str,
    polygon: list,
    gt_date: datetime,
    date_offset_amount: int, 
    date_offset_unit: str="day",
    date_offset_policy: str="both", # (before, after, both)
    collection_name: str="COPERNICUS/S2_SR_HARMONIZED",
    s2_bands: list=['B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9', 'B11','B12','MSK_CLDPRB'],
    drivefolder: str="geeExports",
    drivefolderid='',
    batch: bool=False, # if false, sequential dl o/w parallel
    agg: str=None, # if None, raw images, o/w compute agg
    mosaic: bool=False, # if True mosaic same dates
    dtye: str="uint16",
    verbose: bool=False,
):
    assert date_offset_policy is ["before", "after", "both"], f'Invalid date_offset_policy: {date_offset_policy}. Value must be in {["before", "after", "both"]}.'
    assert date_offset_amout >= 0, f'date_offset_amount must be positive'
    assert agg in [None, "mean", "median", "cloudless_mean", "cloudless_median"], f'Invalid agg: {agg}. Value must be in {[None, "mean", "median"]}.'
    assert dtype in ["uint16"], f'Invalid dtype: {dtype}. Value must be in {["uint16"]}.'
    # Connect to GDrive
    if not drivefolderid and len(drive.ListFile({'q': "title='{id}'".format(id=drivefolder)}).GetList())==0: 
        file_metadata = {'title': drivefolder,
                            'parents': ['root'],
                            'mimeType': 'application/vnd.google-apps.folder'}
        folder = drive.CreateFile(file_metadata)
        folder.Upload()
        drivefolderid = folder['id']
    elif not drivefolderid: 
        drivefolderid = drive.ListFile({'q': "title='{id}'".format(id=drivefolder)}).GetList()[0]['id']  #version v2 of Google Drive API
    
    # polygon
    aoi = ee.Geometry.Polygon(polygon)
    
    # dates
    gt_date = ee.Date(gt_date)
    if date_offset_policy == "before":
        start_date = gt_date.advance(-date_offset_amount, date_offset_unit)
        end_date = gt_date
    elif date_offset_policy == "after":
        start_date = gt_date
        end_date = gt_date.advance(date_offset_amount, date_offset_unit)
    else:
        start_date = gt_date.advance(-date_offset_amount, date_offset_unit)
        end_date = gt_date.advance(date_offset_amount, date_offset_unit)
    
    # image collection
    icol = (ee.ImageCollection(collection_name)
             .filterBounds(aoi)
             .filterDate(start_date, end_date)
             .select(s2_bands))
    if mosaic:
        icol = MergeByDate(icol)
    if agg is not None:
        if "cloudless" in agg:
            # TODO: cloud masking
            raise NotImplementedError()
        agg_img = eval(f"icol.{agg}()")
        download_img(agg_img)
        download_local(driverfolderid, localdir, datetime.now(), verbose=verbose)
    
    # Download
    else:
        now = datetime.now()
        num_tasks = icol.size().getInfo()
        icol_list = icol.toList(num_tasks)
        # Parallel download
        if batch:
            for image in icol_list: 
                download_image(image)
            download_local(driverfolderid, localdir, datetime.now(), verbose=verbose)
        # Sequential download
        else:
            for image in icol_list: 
                download_image(image)
                download_local(driverfolderid, localdir, datetime.now(), verbose=verbose)

In [37]:
type(polygon["coordinates"])

list

In [57]:
ee.Geometry.Polygon(polygon["coordinates"]).getInfo()["coordinates"]

[[[563899.2011781419, 6755834.658824679],
  [539024.1310164346, 6755178.560310203],
  [539248.8400359971, 6754543.159912221],
  [538865.0969036139, 6753917.5781549355],
  [538897.9524304823, 6749568.097829428],
  [539887.1999744953, 6748411.749972747],
  [541210.2036883761, 6746911.441425804],
  [541475.9200962898, 6746610.10985395],
  [541476.1400709585, 6746609.859871178],
  [542336.7099804848, 6745627.459846377],
  [542355.5400050975, 6745595.549843346],
  [542361.4600328822, 6745558.9698804505],
  [542345.7820769094, 6744955.649179959],
  [542317.7199797049, 6743876.039827059],
  [543839.1000907369, 6743876.039930546],
  [543875.3101077093, 6743869.249920703],
  [543906.6100597142, 6743849.819913843],
  [543928.7400657623, 6743820.369920687],
  [543938.7100632838, 6743784.899846158],
  [543993.8001072792, 6743165.399859212],
  [543994.1801328451, 6743155.069907558],
  [543993.280089667, 6743093.79989065],
  [543845.8999475149, 6733081.109922016],
  [547111.2481775562, 6732607.14491

In [ ]:
538396,6.75601e+06, 538147,6.74546e+06


In [39]:
icol = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterBounds(ee.Geometry.Rectangle([, 0.23, 0.24]))\
    .select(["B"])

In [43]:
icol.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'COPERNICUS/S2_SR_HARMONIZED',
 'version': 1683814929787050.0,
 'properties': {'date_range': [1490659200000, 1647907200000],
  'period': 0,
  'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'keywords': ['copernicus',
   'esa',
   'eu',
   'msi',
   'reflectance',
   'sentinel',
   'sr'],
  'system:visualization_0_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/COPERNICUS_S2_SR_thumb.png',
  'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 L2 data are downloaded from scihub. They were\ncomputed by running sen2cor. WARNING: ESA did not produce L2 data\nfor all L1 assets, and earlier L2 coverage is not global.</p><p>The assets contain\n12 UINT16 spectral band

In [45]:
icol.size().getInfo()

0

In [47]:
icol.toList(icol.size())

In [52]:
import argparse
args = argparse.Namespace()
args.value = 10
args.root = "usr/"
vars(args)

{'value': 10, 'root': 'usr/'}